<a href="https://colab.research.google.com/github/jaadu-1/Algo-trading/blob/main/decision_track1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

krish39696_decision_track_path = kagglehub.dataset_download('krish39696/decision-track')
krish39696_submission_path = kagglehub.dataset_download('krish39696/submission')

print('Data source import complete.')


In [ ]:
import os
os.listdir("/kaggle/input")


['decision-track']

In [ ]:
!pip install lightgbm

import pandas as pd
import pyarrow.parquet as pq
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [ ]:
# Base path
base_path = "/kaggle/input/decision-track/"

# Load files
train = pd.read_parquet(base_path + "train_data.parquet")
test = pd.read_parquet(base_path + "test_data.parquet")
offers = pd.read_parquet(base_path + "offer_metadata.parquet")
trans = pd.read_parquet(base_path + "add_trans.parquet")
events = pd.read_parquet(base_path + "add_event.parquet")


In [ ]:
trans.columns


Index(['id2', 'f367', 'f368', 'f369', 'f370', 'f371', 'f372', 'id8', 'f374'], dtype='object')

In [ ]:
print("📦 Train columns:\n", train.columns.tolist())
print("\n📦 Offers columns:\n", offers.columns.tolist())


📦 Train columns:
 ['id1', 'id2', 'id3', 'id4', 'id5', 'y', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', '

In [ ]:
# Only do this if you haven't merged already (and don't have f374/f375/f376)

# Convert id3 to string type
offers['id3'] = offers['id3'].astype(str)

# Select only the columns we want to merge (avoid duplicate fXXXs)
offers_clean = offers[['id3', 'f374', 'f375', 'f376', 'f377', 'f378']]  # adjust based on offers.head()

# Also convert id3 in train and test — if available
if 'id3' in train.columns:
    train['id3'] = train['id3'].astype(str)
    test['id3'] = test['id3'].astype(str)

    # Merge offers
    train = train.merge(offers_clean, on='id3', how='left')
    test = test.merge(offers_clean, on='id3', how='left')
    print("✅ Successfully merged offers metadata into train and test.")
else:
    print("❗ Skipping merge — 'id3' not present in train/test (likely already merged).")


✅ Successfully merged offers metadata into train and test.


In [ ]:
print("🔍 Transactions columns:\n", trans.columns.tolist())


🔍 Transactions columns:
 ['id2', 'f367', 'f368', 'f369', 'f370', 'f371', 'f372', 'id8', 'f374']


In [ ]:
print("🔍 Train columns:\n", train.columns.tolist())
print("\n🔍 Test columns:\n", test.columns.tolist())


🔍 Train columns:
 ['id1', 'id2', 'id3', 'id4', 'id5', 'y', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', '

In [ ]:
# Step 3B: Use f370 as amount and f374 as user ID

# 1. Ensure f370 is numeric
trans['f370'] = pd.to_numeric(trans['f370'], errors='coerce')

# 2. Rename columns for clarity
trans = trans.rename(columns={'id2': 'user_id', 'f370': 'amount', 'f374': 'user_key'})

# 3. Convert to string
trans['user_key'] = trans['user_key'].astype(str)
train['f374'] = train['f374'].astype(str)
test['f374'] = test['f374'].astype(str)

# 4. Aggregate
trans_agg = trans.groupby('user_key')['amount'].agg(['sum', 'mean', 'count']).reset_index()
trans_agg.columns = ['user_key', 'total_spent', 'avg_spent', 'transaction_count']

# 5. Merge into train/test
train = train.merge(trans_agg, left_on='f374', right_on='user_key', how='left')
test = test.merge(trans_agg, left_on='f374', right_on='user_key', how='left')

# 6. Drop redundant merge column
train = train.drop(columns=['user_key'])
test = test.drop(columns=['user_key'])

print("✅ Step 3B complete! Transactions merged successfully.")


✅ Step 3B complete! Transactions merged successfully.


In [ ]:
import numpy as np

# Define columns to exclude (ID columns)
drop_cols = ['f374']  # user_id
target = 'y' if 'y' in train.columns else None

# Step 1: Drop IDs and target
X = train.drop(columns=drop_cols + ([target] if target else []))
X_test = test.drop(columns=drop_cols)

# Step 2: Convert columns individually and fill NAs
def safe_convert(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except:
                df[col] = np.nan  # If conversion fails, set to NaN
    return df.fillna(-1)

X = safe_convert(X)
X_test = safe_convert(X_test)
y = train[target] if target else None

print("✅ Step 4 complete (optimized).")
print(f"X shape: {X.shape}, X_test shape: {X_test.shape}")


✅ Step 4 complete (optimized).
X shape: (770164, 378), X_test shape: (369301, 378)


In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# Split the data (80% train, 20% val)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Prepare LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

print("✅ Step 5 done: Data split and prepared for LightGBM.")
print("Train size:", X_train.shape, "| Validation size:", X_val.shape)



✅ Step 5 done: Data split and prepared for LightGBM.
Train size: (616131, 378) | Validation size: (154033, 378)


In [ ]:
# LightGBM parameters
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'verbose': -1
}

# Train the model using sklearn API (better with early stopping)
model = lgb.LGBMClassifier(**params, n_estimators=1000)

# Fit with early stopping on validation AUC
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

print("✅ Step 6 complete: Model trained.")


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[966]	valid_0's auc: 0.953538
✅ Step 6 complete: Model trained.


In [ ]:
from sklearn.metrics import roc_auc_score

# Predict probabilities for validation set
val_preds = model.predict_proba(X_val)[:, 1]

# Calculate AUC
val_auc = roc_auc_score(y_val, val_preds)
print(f"✅ Validation AUC Score: {val_auc:.4f}")


✅ Validation AUC Score: 0.9535


In [ ]:
# Predict probabilities for test set
test_preds = model.predict_proba(X_test)[:, 1]

# Print a preview of predictions
print("🔍 Sample predictions on test set:")
print(test_preds[:10])  # Show first 10 predictions


🔍 Sample predictions on test set:
[0.00276554 0.02524373 0.98770245 0.00250081 0.005009   0.00288031
 0.00682164 0.00385238 0.03232597 0.00404257]


In [ ]:
# List files and folders under /kaggle/input
import os

print("🗂 Files in /kaggle/input:")
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(f"📁 {dirname}")
    for filename in filenames:
        print(f"   📄 {filename}")


🗂 Files in /kaggle/input:
📁 /kaggle/input
📁 /kaggle/input/decision-track
   📄 test_data.parquet
   📄 add_event.parquet
   📄 data_dictionary.csv
   📄 offer_metadata.parquet
   📄 add_trans.parquet
   📄 train_data.parquet


In [ ]:
# Load your specific submission template
submission = pd.read_csv("/kaggle/input/submission/685404e30cfdb_submission_template.csv")

# Add predictions
submission['y'] = test_preds

# Save final submission file
output_path = "/kaggle/working/my_submission.csv"
submission.to_csv(output_path, index=False)

# Confirm it's saved
print(f"✅ Submission file saved at {output_path}")
submission.head()


✅ Submission file saved at /kaggle/working/my_submission.csv


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id1,id2,id3,id5,pred,y
0,1000061_31794_16-23_2023-11-05 09:47:54.852,1000061,31794,11/5/2023,NaN,0.002766
1,1000061_16099_16-23_2023-11-05 09:28:11.514,1000061,16099,11/5/2023,NaN,0.025244
2,1000061_72292_16-23_2023-11-05 09:28:07.697,1000061,72292,11/5/2023,NaN,0.987702
3,1000061_27945_16-23_2023-11-05 09:28:04.157,1000061,27945,11/5/2023,NaN,0.002501
4,1000061_944713_16-23_2023-11-05 09:28:12.529,1000061,944713,11/5/2023,NaN,0.005009


In [ ]:
import os

print("Files in /kaggle/working:")
print(os.listdir("/kaggle/working"))


Files in /kaggle/working:
['my_submission.csv', '.virtual_documents']
